# 2.3.6 General Solution Method

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/119NlZyqnemGLh_GChesinxdAayry6sU-?usp=sharing)

We have seen in our graphical solutions that, if an optimal solution exists, it occurs at an *extreme point* of the feasible region. This fundamental property of linear programming problems is the foundation for a general solution method called the **Simplex method**. Because only the finitely many extreme points need be examined (rather than all the points in the feasible region), an optimal solution may be found systematically by considering the objective function values at the extreme points. In fact, in actual practice, only a small subset of the extreme points need be examined. The following sections will demonstrate how the Simplex method is able to locate optimal solutions with such efficiency.

**Interactive Example**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import ipywidgets as widgets
from IPython.display import display

def find_intersection(line1, line2):
    A = np.array([[line1[0], line1[1]], [line2[0], line2[1]]])
    b = np.array([line1[2], line2[2]])
    if np.linalg.det(A) == 0:
        return None  # Parallel lines, no intersection
    return np.linalg.solve(A, b)

def is_feasible(point, constraints):
    x, y = point
    for a, b, c, sign in constraints:
        if sign == '<' and not (a * x + b * y <= c):
            return False
        if sign == '>' and not (a * x + b * y >= c):
            return False
    return True

def find_optimal_points(obj_type, a, b, constraints):
    feasible_points = []
    for (line1, line2) in combinations(constraints, 2):
        intersection = find_intersection(line1, line2)
        if intersection is not None and is_feasible(intersection, constraints):
            feasible_points.append(intersection)

    feasible_points = np.array(feasible_points)
    if feasible_points.size == 0:
        return np.array([]), None

    obj_values = [a * x + b * y for x, y in feasible_points]
    optimal_index = np.argmax(obj_values) if obj_type == "maximize" else np.argmin(obj_values)
    optimal_point = feasible_points[optimal_index]
    return feasible_points, optimal_point

def plot_feasible_region_and_objective(constraints, a, b, obj_type, x_range, y_range):
    x, y = np.meshgrid(x_range, y_range)
    feasible_region = np.ones_like(x, dtype=bool)

    for a_c, b_c, c_c, sign in constraints:
        if sign == '<':
            feasible_region &= (a_c * x + b_c * y <= c_c)
        elif sign == '>':
            feasible_region &= (a_c * x + b_c * y >= c_c)

    plt.imshow(feasible_region.astype(int), extent=(x.min(), x.max(), y.min(), y.max()), origin="lower", cmap="Greys", alpha=0.3)

    for a_c, b_c, c_c, sign in constraints:
        y_vals = np.linspace(y_range.min(), y_range.max(), 200)
        if a_c != 0:
            x_vals = (c_c - b_c * y_vals) / a_c
        else:
            x_vals = np.full_like(y_vals, c_c / b_c)
        plt.plot(x_vals, y_vals, label=f'{a_c}x + {b_c}y {sign} {c_c}')

    feasible_points, optimal_point = find_optimal_points(obj_type, a, b, constraints)
    if feasible_points.size > 0:
        feasible_x, feasible_y = zip(*feasible_points)
        plt.scatter(feasible_x, feasible_y, color='blue', label='Feasible Points')

    if optimal_point is not None:
        plt.scatter(*optimal_point, color='red', s=100, label='Optimal Point', edgecolors='black')

    obj_x = np.linspace(x_range.min(), x_range.max(), 200)
    obj_y = (-a * obj_x) / b if b != 0 else np.full_like(obj_x, 0)
    plt.plot(obj_x, obj_y, '--r', label='Objective Function')

    plt.xlim(x_range.min(), x_range.max())
    plt.ylim(y_range.min(), y_range.max())
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.legend(loc='upper right')
    plt.title(f"{obj_type.capitalize()} Objective Function")
    plt.grid(True)
    plt.show()

def update_plot(obj_type, a, b, *constraint_values):
    num_constraints = len(constraint_values) // 4
    constraints = [(constraint_values[i * 4], constraint_values[i * 4 + 1], constraint_values[i * 4 + 2], constraint_values[i * 4 + 3]) for i in range(num_constraints)]

    x_range = np.linspace(0, 16, 300)
    y_range = np.linspace(0, 10, 300)
    plt.figure(figsize=(8, 8))
    plot_feasible_region_and_objective(constraints, a, b, obj_type, x_range, y_range)

def interactive_solver():
    obj_type = widgets.Dropdown(
        options=['maximize', 'minimize'],
        description='Objective:',
    )

    a_slider = widgets.FloatSlider(value=1, min=-20, max=20, step=1, description='a (x):')
    b_slider = widgets.FloatSlider(value=1, min=-20, max=20, step=1, description='b (y):')

    num_constraints = widgets.Dropdown(options=[1, 2, 3, 4, 5, 6], description='Constraints:')

    constraint_sliders = []
    def update_constraints(*args):
        nonlocal constraint_sliders
        for slider in constraint_sliders:
            slider.close()
        constraint_sliders.clear()

        for i in range(num_constraints.value):
            a_s = widgets.FloatSlider(value=1, min=-20, max=20, step=1, description=f'a{i+1}:')
            b_s = widgets.FloatSlider(value=1, min=-20, max=20, step=1, description=f'b{i+1}:')
            c_s = widgets.FloatSlider(value=5, min=-20, max=20, step=1, description=f'c{i+1}:')
            sign_s = widgets.Dropdown(options=['<', '>'], description=f'sign{i+1}:')
            constraint_sliders.extend([a_s, b_s, c_s, sign_s])
        display_controls()

    num_constraints.observe(update_constraints, names='value')

    def display_controls():
        ui = widgets.VBox([obj_type, a_slider, b_slider, num_constraints] + constraint_sliders)
        output = widgets.Output()

        def update(_):
            with output:
                output.clear_output(wait=True)
                update_plot(obj_type.value, a_slider.value, b_slider.value, *[s.value for s in constraint_sliders])

        for slider in [a_slider, b_slider, num_constraints] + constraint_sliders + [obj_type]:
            slider.observe(update, names='value')

        display(ui, output)
        update(None)

    update_constraints()

interactive_solver()

Output()